#4.1 선형회귀

**선형회귀**는 데이터의 경향성을 가장 잘 설명하는 하나의 직선을 예측하는 것


In [ ]:
#2018년 우리나라의 지역별 인구증가율과 고령인구비율 데이터를 시각화
import tensorflow as tf
import matplotlib.pyplot as plt

population_inc = [0.3, -0.78, 1.26, 0.03, 1.11, 15.17, 0.24, -0.24, -0.47, -0.77, -0.37,-0.85, -0.41, -0.27, 0.02, -0.76, 2.66]
population_old = [12.27, 14.44, 11.87, 18.75, 17.52, 9.29, 16.37, 19.78, 19.51, 12.65, 14.74, 10.72, 21.94, 12.83, 15.51, 17.14, 14.42]


plt.plot(population_inc, population_old, 'bo')
plt.xlabel('Population growth Rate')
plt.ylabel('Elderly Population Rate')
plt.show()

여기서 오른쪽 아래에 치우친 하나의 점을 '극단치'라고 부른다. 일반적인 경향에서 벗어나는 사례를 칭한다. 데이터의 일반적인 경향을 알아보기 위해서 극단치는 제거하도록 한다. 

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

population_inc = [0.3, -0.78, 1.26, 0.03, 1.11, 15.17, 0.24, -0.24, -0.47, -0.77, -0.37,-0.85, -0.41, -0.27, 0.02, -0.76, 2.66]
population_old = [12.27, 14.44, 11.87, 18.75, 17.52, 9.29, 16.37, 19.78, 19.51, 12.65, 14.74, 10.72, 21.94, 12.83, 15.51, 17.14, 14.42]

population_inc = population_inc[:5] + population_inc[6:]
population_old = population_old[:5] + population_old[6:]
plt.plot(population_inc, population_old, 'bo')
plt.xlabel('Population growth Rate')
plt.ylabel('Elderly Population Rate')
plt.show()

이제 일반적인 데이터를 토대로 선형 회귀를 해본다.

데이터의 경향성을 가장 잘 설명하는 하나의 직선과 각 데이터의 차이를 **잔차**라고 한다.

이 잔차의 제곱을 최소화하는 알고리즘을 **최소제곱법**이라고 한다.

이를 직접 계산해서 해보자면

In [ ]:
import numpy as np

X = population_inc
Y = population_old

#X, Y의 평균을 구한다.
x_bar = sum(X) / len(X)
y_bar = sum(Y) / len(Y)

#최소제곱법으로 a,b를 구한다.
#두 개 이상의 리스트를 하나로 묶는 list(zip(list_1, list_2))를 사용한다.
a = sum([(y - y_bar) * (x - x_bar) for y,x in list(zip(Y,X))])
a /= sum([(x - x_bar) ** 2 for x in X])
b  = y_bar - a * x_bar
print('a:', a, 'b:', b)

#그래프를 그리기 위해 회귀선의 x, y 데이터를 구한다.
line_x = np.arange(min(X), max(X), 0.01) #numpy의 arange를 사용하는 이유는 range는 정수로만 구성되기 때문에 실숫값을 사용하기 위해서이다.
line_y = a * line_x + b

#붉은색 실선으로 회귀선을 그린다
plt.plot(line_x, line_y, 'r-')

plt.plot(X, Y, 'bo')
plt.xlabel('Population Growth Rate')
plt.ylabel('Elderly Population Rate')
plt.show()

위와 같이 복잡한 수식과 최소제곱법을 쓰지 않고도 텐서플로를 이용하면 회귀선을 그릴 수 있다.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

# a,b를 랜덤한 값으로 초기화
a = tf.Variable(random.random())
b = tf.Variable(random.random())

# 잔차의 제곱의 평균을 반환하는 함수
def compute_loss():
    y_pred = a * X + b
    # 기대 출력인 Y에서 실제 출력인 pred_y를 빼는데, 이를 잔차라고 부른다.
    loss = tf.reduce_mean((Y - y_pred) ** 2) #잔차의 제곱 평균을 loss로 반환한다.
    return loss

optimizer = tf.optimizers.Adam(lr=0.07)
for i in range(1000):
    # 잔차의 제곱의 평균을 최소화 한다(minimize)
    optimizer.minimize(compute_loss, var_list = [a,b])

    if i% 100 == 99:
        print( i, 'a:', a.numpy(), 'b:', b.numpy(), 'loss:', compute_loss().numpy())

line_x = np.arange(min(X), max(X), 0.01)
line_y = a * line_x + b

plt.plot(line_x, line_y, 'r-')
plt.plot(X, Y, 'bo')
plt.show()

# 4.2 다항회귀

**비선형회귀**는 선형 회귀로 표현할 수 없는 데이터의 경향성을 설명하기 위한 회귀

직선 ax + b가 아니라 ax^2 + bx + c와 같은 함수로 회귀선을 써보는 것이다.

위의 선형회귀 선을 조금 고쳐서 사용한다.

In [ ]:
#a,b,c를 랜덤한 값으로 초기화한다.
a = tf.Variable(random.random())
b = tf.Variable(random.random())
c = tf.Variable(random.random())

# 잔차의 제곱의 평균을 반환하는 함수
def compute_loss():
    y_pred = a * X * X + b * X + c
    loss = tf.reduce_mean((Y - y_pred) ** 2)
    return loss

optimizer = tf.optimizers.Adam(lr=0.07)
for i in range(1000):
    # 잔차의 제곱의 평균을 최소화 한다(minimize)
    optimizer.minimize(compute_loss, var_list = [a,b,c])

    if i% 100 == 99:
        print( i, 'a:', a.numpy(), 'b:', b.numpy(), 'loss:', compute_loss().numpy())

line_x = np.arange(min(X), max(X), 0.01)
line_y = a * line_x * line_x + b * line_x + c

plt.plot(line_x, line_y, 'r-')
plt.plot(X, Y, 'bo')
plt.show()

ax^3 + bx^2 + cx + d로 바꾸어서 더 복잡한 다항회귀도 가능하다.

In [ ]:
# 3차 다항회귀

#a,b,c를 랜덤한 값으로 초기화한다.
a = tf.Variable(random.random())
b = tf.Variable(random.random())
c = tf.Variable(random.random())
d = tf.Variable(random.random())


# 잔차의 제곱의 평균을 반환하는 함수
def compute_loss():
    y_pred = a * X * X * X + b * X * X + c * X + d
    loss = tf.reduce_mean((Y - y_pred) ** 2)
    return loss

optimizer = tf.optimizers.Adam(lr=0.07)
for i in range(1000):
    # 잔차의 제곱의 평균을 최소화 한다(minimize)
    optimizer.minimize(compute_loss, var_list = [a,b,c,d])

    if i% 100 == 99:
        print( i, 'a:', a.numpy(), 'b:', b.numpy(), 'c:', c.numpy(), 'd:', d.numpy(), 'loss:', compute_loss().numpy())

line_x = np.arange(min(X), max(X), 0.01)
line_y = a * line_x * line_x * line_x + b * line_x * line_x + c * line_x + d

plt.plot(line_x, line_y, 'r-')
plt.plot(X, Y, 'bo')
plt.show()

# 4.2 딥러닝 네트워크를 이용한 회귀

In [ ]:
import tensorflow as tf
import numpy as np


X = [0.3, -0.78, 1.26, 0.03, 1.11, 0.24, -0.24, -0.47, -0.77, -0.37,-0.85, -0.41, -0.27, 0.02, -0.76, 2.66]
Y = [12.27, 14.44, 11.87, 18.75, 17.52, 16.37, 19.78, 19.51, 12.65, 14.74, 10.72, 21.94, 12.83, 15.51, 17.14, 14.42]

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=6, activation='tanh', input_shape=(1,)), #첫번째 활성함수로 tanh사용.
                                                                                                  #뉴런은 6개를 할당. 뉴런은 많을수록 딥러닝의 네트워크 표현력이 좋아지는 건 맞지만 과적합이 일어날 수 있으니 주의
                             tf.keras.layers.Dense(units=1) # X값 하나에 Y값 하나를 출력해야 하므로 뉴런은 1개 할당
])

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1), loss='mse')

model.summary()



In [ ]:
model.fit(X, Y, epochs=10)

X를 입력하면 Y가 정답이 되도록 10회 학습시킨다.

손실에 변화가 없으면 학습이 거의 다 된것을 의미한다.

In [ ]:
model.predict(X)

In [ ]:
line_x = np.arange(min(X), max(X), 0.01)
line_y = model.predict(line_x)

plt.plot(line_x, line_y, 'r-')
plt.plot(X, Y, 'bo')
plt.show()

#4.4 보스턴 주택 가격 데이터 센트

보스턴 주택 가격 데이터 세트를 이용한 주택 가격예측 네트워크를 만든다.

훈련데이터는 학습 과정에 사용되는 데이터이고, 테스트 데이터는 학습 결과를 평가하기 위한 데이터이다.

학습할 때는 훈련 데이터만 사용하고, 테스트 데이터는 볼 수 없다.

훈련 데이터로 학습할 때 일부 데이터를 떼어내 검증데이터로 만들 수 있다.

이는 학습이 잘 되고 있는지 검증하는 용도로 쓰이며, 성적이 잘 나오지 않는다면 학습을 중단할 수 있다.

딥러닝 네트워크 가중치에 영향을 주는 데이터는 훈련데이터 뿐이다.

In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_X, train_Y), (test_X, test_Y) = boston_housing.load_data()

print(len(train_X), len(test_X))
print(train_X[0])
print(train_Y[0])

딥러닝에서는 데이터를 전처리해서 **정규화**해야 학습 효율이 좋다.

데이터를 정규화하려면 각 데이터에서 평균값을 뺀 다음 표준편차로 나눈다.

이는 데이터의 분포를 정규분포로 옮기는 역할을 한다.

In [ ]:
x_mean = train_X.mean()
x_std = train_X.std()
#정규화
train_X -= x_mean
train_X /= x_std
test_X -= x_mean
test_X /= x_std

y_mean = train_Y.mean()
y_std = train_Y.std()
#정규화
train_Y -= y_mean
train_Y /= y_std
test_Y -= y_mean
test_Y /= y_std

print(len(train_X))
print(train_Y[0])


In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=52, activation='relu', input_shape=(13,)),#위에서보다 레이어에 들어가는 뉴런의 수가 증가함.
                             tf.keras.layers.Dense(units=39, activation='relu'),                   #레이어의 수와 은닉층의 뉴런 수를 늘리면 모델의 표현력이 좋아진다.  
                             tf.keras.layers.Dense(units=26, activation='relu'),
                             tf.keras.layers.Dense(units=1)
])
#ReLU의 경우 지금처럼 여러개의 레이어를 겹쳐서 사용할 때 시그모이드 함수나 tanh보다 좋은 결과를 얻을 수 있다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='mse')

model.summary()

In [ ]:
#model.fit함수로 회귀모델 학습
#epochs = 학습 횟수, batch_size = 한번에 학습시키는 데이터의 수, validation_split = 훈련데이터 중 검증데이터로 사용하는 비율
history = model.fit(train_X, train_Y, epochs=25, batch_size = 32, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()
plt.show()

파란색이 데이터의 손실, 붉은 색이 검증 데이터의 손실이다. 

데이터의 손실은 감소하지만 검증 데이터의 손실은 줄지 않고 유지하거나 증가한다. 

In [ ]:
model.evaluate(test_X, test_Y)

In [ ]:
pred_Y = model.predict(test_X)

plt.figure(figsize=(5,5))
plt.plot(test_Y, pred_Y, 'b.')
plt.axis([min(test_Y), max(test_Y), min(test_Y), max(test_Y)])

#y=x에 해당하는 대각선
plt.plot([min(test_Y), max(test_Y)], [min(test_Y), max(test_Y)], ls="--", c=".3")
plt.show()

실제 주택 가격 (test_Y)에 대해 예측 주택 가격(pred_Y)이 일정 값에 머물러 있는 것 같다. 이상적이라면 그래프를 가로지르는 대각선에 모든 점이 위치해야한다.

검증 데이터에 대한 성적이 좋아지면 테스트 데이터에 대한 성적도 좋아질 것이다. 검증 데이터에 대한 성적이 좋아지려면 val_loss가 높아지지 않도록, 즉 네트워크가 훈련 데이터에 과적합 되지 않도록 학습 도중에 끼어들어 학습을 멈춰야한다.

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=52, activation='relu', input_shape=(13,)),
                             tf.keras.layers.Dense(units=39, activation='relu'),
                             tf.keras.layers.Dense(units=26, activation='relu'),
                             tf.keras.layers.Dense(units=1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='mse')

history = model.fit(train_X, train_Y, epochs=25, batch_size=32, validation_split=0.25, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')])

위 함수에 들어간 tf.keras.callbacks.EarlyStopping은 patience는 몇번의 에포크를 기준으로 삼을 것인지, monitor는 어떤 값을 지켜볼 것인디에 대한 인수

val_loss가 3회의 에포크를 수행하는 동안 최고 기록을 갱신하지 못하면 학습을 멈추게 한다.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_X, test_Y)

0.6668정도로 낮은 수치가 나왔다.

실제 주택 가격과 예측 주택 가격을 1:1로 시각화 해본다면

In [ ]:
pred_Y = model.predict(test_X)

plt.figure(figsize=(5,5))
plt.plot(test_Y, pred_Y, 'b.')
plt.axis([min(test_Y), max(test_Y), min(test_Y), max(test_Y)])

plt.plot([min(test_Y), max(test_Y)],[min(test_Y), max(test_Y)], ls="--", c=".3")
plt.show()